# Particle Image Velocimetry (PIV) Analysis Workflow for xCELLigence timelapse image data #

### Notebook by Joanna W. Pylvänäinen, [Cell Migration lab](https://cellmig.org/), Turku Finland
---

This notebook provides a workflow for Particle Image Velocimetry (PIV) analysis for xCELLigence timelapse data. It enables the processing of multi-frame TIFF videos to compute and analyze flow metrics.

**Expected input data:**
- Time-lapse tiff
- RGB images (will be converted to grayscale)

**Workflow outputs:**
- Flow field visualizations: Color-coded flow vectors between each frame.
- Summary flow field visualizations for user defined frame interval.
- Summary flow field visualizations for the whole video.
- CSV metrics: Average and max velocities, average flow direction and divergence.
- Individual plots for each video
- Comparison plots: User defined plots for selected video combinations.
- Averaged comparison plots: User defined plots for selected condition combinations.

**If you use this tool please cite the original code**
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.4409178.svg)](https://doi.org/10.5281/zenodo.4409178)

In [ ]:
# @title 1.0 Install dependencies

# Force reinstall NumPy with a stable version
!pip install --upgrade numpy
!pip install opencv-python matplotlib openpiv tifffile natsort scipy

import os
os.kill(os.getpid(), 9)

In [ ]:
# @title 1.1 Load dependencies

# Import libraries
import os
import cv2
import numpy as np #NumPy is imported here
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
from scipy.ndimage import map_coordinates #map_coordinates import after numpy
from openpiv.pyprocess import extended_search_area_piv
from openpiv.validation import sig2noise_val
from openpiv.filters import replace_outliers
from openpiv.scaling import uniform
import matplotlib.cm as cm
import pandas as pd
import gc
import glob
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.ndimage import uniform_filter1d
import re
from natsort import natsorted






## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.


<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/content/gdrive')
%cd /gdrive

In [ ]:
# @title 2.1 Set input and output paths
#@markdown ###Expected file format: timelapse .tif, RGB image


input_folder = '/content/gdrive/MyDrive/Collab-Megan/Test_data'  # @param {type: "string"}
output_folder = '/content/gdrive/MyDrive/Collab-Megan/Test_data_results'  # @param {type: "string"}

# Count the number of files
file_count = len([f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))])

print(f"Number of videos in the folder: {file_count}")

#input_folder = '/content/drive/Shareddrives/PIV-analysis/Jammindata2_for_analysis_A-B/'  # Input folder
#output_folder = '/content/drive/Shareddrives/PIV-analysis/Jammindata2_results_A-B/'  # Output folder
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# @title 3.1 PIV analysis
#@markdown If you have multiple vidos with hign number of frames, consider processing in batches to avoid running out of RAM
#@markdown ###Calibrate your data
pixel_size_um = 1.73  # @param {type:"number"}
time_interval_min = 20  # @param {type:"number"}
#@markdown ###Define parameters for the PIV analysis
window_size = 32 # @param {type:"number"}
overlap = 16 # @param {type:"number"}
search_area_size = 32 # @param {type:"number"}
summary_interval = 10  # @param {type:"number"}

# Define functions
def load_tiff_video(file_path):
    """
    Load a multi-frame TIFF video as a list of frames.
    """
    print(f"Loading: {file_path}")
    tiff_frames = imread(file_path)
    if len(tiff_frames.shape) == 4:  # Multi-frame RGB TIFF
        return [frame for frame in tiff_frames]
    elif len(tiff_frames.shape) == 3:  # Multi-frame grayscale
        return [tiff_frames[i] for i in range(tiff_frames.shape[0])]
    else:
        raise ValueError(f"Unexpected TIFF format with shape {tiff_frames.shape}")

def convert_to_grayscale(image):
    if len(image.shape) == 3:  # RGB image
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    elif len(image.shape) == 2:  # Already grayscale
        return image
    else:
        raise ValueError(f"Unexpected image format with shape {image.shape}")

def piv_analysis(image1, image2):
    assert len(image1.shape) == 2 and len(image2.shape) == 2, "Images must be 2D arrays."
    #window_size = 32
    #overlap = 16
    #search_area_size = 32

    u, v, sig2noise = extended_search_area_piv(
        image1.astype(np.int32),
        image2.astype(np.int32),
        window_size=window_size,
        overlap=overlap,
        dt=1,
        search_area_size=search_area_size,
    )
    flags = sig2noise > 1.3
    u, v = replace_outliers(u, v, flags, method='localmean', max_iter=3, kernel_size=2)
    x, y = np.meshgrid(
        np.arange(0, u.shape[1]) * window_size + window_size / 2,
        np.arange(0, u.shape[0]) * window_size + window_size / 2
    )
    return x, y, u, v

def visualize_colored_flow_field(x, y, u, v, image_shape, save_path=None):
    angles = np.arctan2(v, u)
    norm = plt.Normalize(vmin=-np.pi, vmax=np.pi)
    colors = cm.hsv(norm(angles))
    fig, ax = plt.subplots(figsize=(image_shape[1] / 100, image_shape[0] / 100), dpi=100)
    ax.set_xlim(0, image_shape[1])
    ax.set_ylim(image_shape[0], 0)
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            ax.arrow(
                x[i, j], y[i, j], u[i, j], v[i, j],
                color=colors[i, j], head_width=3, head_length=3
            )
    if save_path:
        fig.canvas.draw()
        image_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype='uint8')
        image_array = image_array.reshape(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), 4)
        imwrite(save_path, image_array[..., :3])
        plt.close(fig)


# Updated loop for sequential file handling
for filename in sorted(os.listdir(input_folder)):
    if filename.endswith('.tiff') or filename.endswith('.tif'):
        video_name = os.path.splitext(filename)[0]
        video_path = os.path.join(input_folder, filename)

        print(f"Processing video: {video_name}")

        # Load and preprocess the video
        frames = load_tiff_video(video_path)
        grayscale_frames = [convert_to_grayscale(frame) for frame in frames]
        video_output_folder = os.path.join(output_folder, video_name)
        os.makedirs(video_output_folder, exist_ok=True)

        # Subfolders for flow field images and summary intervals
        frame_pairs_folder = os.path.join(video_output_folder, "frame_pairs")
        os.makedirs(frame_pairs_folder, exist_ok=True)
        summary_intervals_folder = os.path.join(video_output_folder, "summary_intervals")
        os.makedirs(summary_intervals_folder, exist_ok=True)

        # Initialize metrics and cumulative flow variables
        metrics_list = []
        cumulative_u, cumulative_v = None, None
        frame_counter = 0
        overall_u, overall_v = None, None

        for i in range(len(grayscale_frames) - 1):
            img1 = grayscale_frames[i]
            img2 = grayscale_frames[i + 1]
            x, y, u_raw, v_raw = piv_analysis(img1, img2)

            # Compute calibrated values for metrics
            u_calibrated = u_raw * pixel_size_um / time_interval_min
            v_calibrated = v_raw * pixel_size_um / time_interval_min
            flow_magnitudes_calibrated = np.sqrt(u_calibrated**2 + v_calibrated**2)

            # Compute metrics
            avg_velocity = flow_magnitudes_calibrated.mean()
            avg_direction = np.arctan2(v_calibrated, u_calibrated).mean()
            max_magnitude = flow_magnitudes_calibrated.max()
            divergence = np.gradient(u_calibrated, axis=1).mean() + np.gradient(v_calibrated, axis=0).mean()
            end_time_point = (i + 1) * time_interval_min

            metrics_list.append({
                "Time Point": f"{i+1}-{i+2}",
                "End Time Point (min)": end_time_point,
                "Avg Velocity (µm/min)": avg_velocity,
                "Avg Direction (degrees)": np.degrees(avg_direction),
                "Max Flow Magnitude (µm/min)": max_magnitude,
                "Divergence (1/min)": divergence
            })

            # Update cumulative raw flows for visualization
            if overall_u is None:
                overall_u, overall_v = u_raw, v_raw
            else:
                overall_u += u_raw
                overall_v += v_raw

            if cumulative_u is None:
                cumulative_u, cumulative_v = u_raw, v_raw
            else:
                cumulative_u += u_raw
                cumulative_v += v_raw

            # Save individual flow field visualization
            flow_field_path = os.path.join(frame_pairs_folder, f"flow_{i+1}-{i+2}.tiff")
            visualize_colored_flow_field(x, y, u_raw, v_raw, img1.shape, save_path=flow_field_path)

            # Clear memory for individual flow fields
            del u_raw, v_raw
            gc.collect()

            # Save cumulative flow at specified interval
            frame_counter += 1
            if frame_counter == summary_interval or i == len(grayscale_frames) - 2:
                summary_path = os.path.join(summary_intervals_folder, f"summary_flow_{i+1}.tiff")
                visualize_colored_flow_field(x, y, cumulative_u, cumulative_v, img1.shape, save_path=summary_path)
                cumulative_u, cumulative_v = None, None  # Reset cumulative flow
                frame_counter = 0
                gc.collect()

        # Save metrics to CSV
        metrics_df = pd.DataFrame(metrics_list)
        summary_csv_path = os.path.join(video_output_folder, f"{video_name}_flow_summary.csv")
        metrics_df.to_csv(summary_csv_path, index=False)
        print(f"Metrics saved to: {summary_csv_path}")

        # Save overall summary flow field
        summary_image_path = os.path.join(video_output_folder, "summary_flow_field.tiff")
        visualize_colored_flow_field(x, y, overall_u, overall_v, grayscale_frames[0].shape, save_path=summary_image_path)
        print(f"Summary flow field saved to: {summary_image_path}")

        # Clear memory for the next file
        del frames, grayscale_frames, cumulative_u, cumulative_v, metrics_df, overall_u, overall_v
        gc.collect()
        print(f"Memory cleared for: {video_name}")

def concatenate_csv_files(input_folder, output_folder, output_filename="combined_results.csv"):
    """
    Concatenate all CSV files in the input folder and its subfolders into a single CSV file.

    Parameters:
    - input_folder: Path to the folder containing individual CSV files in subfolders.
    - output_folder: Path to the folder where the combined CSV file will be saved.
    - output_filename: Name of the output CSV file.
    """
    # Find all CSV files in the input folder and its subfolders
    csv_files = glob.glob(os.path.join(output_folder, "**", "*_flow_summary.csv"), recursive=True)

    if not csv_files:
        print(f"No CSV files found in the output_folder: {output_folder}")
        return

    # Debug: List found files
    print(f"Found {len(csv_files)} CSV files:")
    for file in csv_files:
        print(f" - {file}")

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    combined_df = pd.DataFrame()  # Initialize an empty DataFrame

    for csv_file in csv_files:
        try:
            # Extract the CSV file name (without the folder path)
            csv_file_name = os.path.basename(csv_file)

            # Read the CSV file
            metrics_df = pd.read_csv(csv_file)

            # Add a column to track the source file name
            metrics_df["CSV File Name"] = csv_file_name

            # Concatenate the data
            combined_df = pd.concat([combined_df, metrics_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {csv_file}: {e}")

    # Save the combined DataFrame to a single CSV file
    output_path = os.path.join(output_folder, output_filename)
    combined_df.to_csv(output_path, index=False)
    print(f"Combined results saved to: {output_path}")


# Example usage

concatenate_csv_files(input_folder, output_folder, output_filename="combined_results.csv")


In [ ]:
# @title Select graphs to plot for each video
plot_avg_velocity = True  # @param {type: "boolean"}
plot_avg_direction = True  # @param {type: "boolean"}
plot_max_flow_magnitude = True  # @param {type: "boolean"}
plot_divergence = True  # @param {type: "boolean"}

def plot_metrics_from_csv(input_folder, output_folder):
    """
    Generate improved plots for each video based on saved CSV files.

    Parameters:
    - input_folder: Folder containing the CSV files.
    - output_folder: Folder to save plots.
    - plot_avg_velocity: Boolean to plot Average Velocity.
    - plot_avg_direction: Boolean to plot Average Direction.
    - plot_max_flow_magnitude: Boolean to plot Maximum Flow Magnitude.
    - plot_divergence: Boolean to plot Divergence.
    """
    csv_files = glob.glob(os.path.join(output_folder, "**", "*_flow_summary.csv"), recursive=True)

    if not csv_files:
      print("No CSV files found.")
      return

    for csv_file in csv_files:
        video_name = os.path.basename(csv_file).replace("_flow_summary.csv", "")
        metrics_df = pd.read_csv(csv_file)

        video_output_folder = os.path.join(output_folder, video_name)
        os.makedirs(video_output_folder, exist_ok=True)

        # Use "End Time Point (min)" as x-axis
        x_values = metrics_df["End Time Point (min)"]

        if plot_avg_velocity:
            plt.figure(figsize=(10, 5))
            plt.plot(x_values, metrics_df["Avg Velocity (µm/min)"], marker='o', color='#9467bd', label='Avg Velocity')
            plt.title(f"Average Velocity - {video_name}", fontsize=16)
            plt.xlabel("Time (min)", fontsize=14)
            plt.ylabel("Average Velocity (µm/min)", fontsize=14)
            plt.xticks(fontsize=12)
            plt.yticks(fontsize=12)
            plt.legend(fontsize=12)
            avg_velocity_path = os.path.join(video_output_folder, f"{video_name}_avg_velocity.png")
            plt.savefig(avg_velocity_path, bbox_inches='tight')
            plt.close()
            print(f"Avg Velocity plot saved to: {avg_velocity_path}")

        if plot_avg_direction:
            plt.figure(figsize=(10, 5))
            plt.plot(x_values, metrics_df["Avg Direction (degrees)"], marker='s', color='#ff7f0e', label='Avg Direction (degrees)')
            plt.title(f"Average Flow Direction - {video_name}", fontsize=16)
            plt.xlabel("Time (min)", fontsize=14)
            plt.ylabel("Average Direction (degrees)", fontsize=14)
            plt.xticks(fontsize=12)
            plt.yticks(fontsize=12)
            plt.legend(fontsize=12)
            avg_direction_path = os.path.join(video_output_folder, f"{video_name}_avg_direction.png")
            plt.savefig(avg_direction_path, bbox_inches='tight')
            plt.close()
            print(f"Avg Direction plot saved to: {avg_direction_path}")

        if plot_max_flow_magnitude:
            plt.figure(figsize=(10, 5))
            plt.plot(x_values, metrics_df["Max Flow Magnitude (µm/min)"], marker='^', color='#2ca02c', label='Max Flow Magnitude')
            plt.title(f"Maximum Flow Magnitude - {video_name}", fontsize=16)
            plt.xlabel("Time (min)", fontsize=14)
            plt.ylabel("Max Flow Magnitude (µm/min)", fontsize=14)
            plt.xticks(fontsize=12)
            plt.yticks(fontsize=12)
            plt.legend(fontsize=12)
            max_flow_magnitude_path = os.path.join(video_output_folder, f"{video_name}_max_flow_magnitude.png")
            plt.savefig(max_flow_magnitude_path, bbox_inches='tight')
            plt.close()
            print(f"Max Flow Magnitude plot saved to: {max_flow_magnitude_path}")

        if plot_divergence:
            plt.figure(figsize=(10, 5))
            plt.plot(x_values, metrics_df["Divergence (1/min)"], marker='d', color='#d62728', label='Divergence')
            plt.title(f"Divergence Over Time - {video_name}", fontsize=16)
            plt.xlabel("Time (min)", fontsize=14)
            plt.ylabel("Divergence (1/min)", fontsize=14)
            plt.xticks(fontsize=12)
            plt.yticks(fontsize=12)
            plt.axhline(0, color='black', linestyle='--', linewidth=1, label='Zero Divergence')
            plt.legend(fontsize=12)
            divergence_path = os.path.join(video_output_folder, f"{video_name}_divergence.png")
            plt.savefig(divergence_path, bbox_inches='tight')
            plt.close()
            print(f"Divergence plot saved to: {divergence_path}")


plot_metrics_from_csv(input_folder, output_folder)


In [ ]:
# @title Map conditions of your data

def map_conditions_and_save(output_path):
    """
    Map conditions to identifiers and save the updated compiled CSV file.

    Parameters:
    - output_path: Path to the combined CSV file containing the "CSV File Name" column.
    """
    try:
        # Load the compiled CSV file
        compiled_data = pd.read_csv(output_path)

        # Extract identifiers like A4 or A4_1W from the file name
        compiled_data['Identifier'] = compiled_data['CSV File Name'].str.extract(r'^([A-Za-z0-9]+(?:_[0-9]+[A-Za-z])?)')
        unique_identifiers = compiled_data['Identifier'].dropna().unique()

        # Sort identifiers naturally
        unique_identifiers = natsorted(unique_identifiers)

        # Create widgets for each identifier
        identifier_widgets = {
            identifier: widgets.Text(
                value='',
                placeholder='Enter condition name',
                description=f'{identifier}:',
                style={'description_width': 'initial'}
            )
            for identifier in unique_identifiers
        }

        # Create a button to submit the mapping
        submit_button = widgets.Button(description="Submit Mapping")
        output = widgets.Output()

        def on_submit_button_click(b):
            with output:
                clear_output()
                print("Condition Mapping Results:")

                # Create a mapping from the user inputs
                condition_mapping = {
                    identifier: widget.value
                    for identifier, widget in identifier_widgets.items()
                }
                print(condition_mapping)

                # Add the conditions to the compiled data
                compiled_data['Condition'] = compiled_data['Identifier'].map(condition_mapping)

                # Save the updated data
                compiled_data.to_csv(output_path, index=False)
                print(f"Updated compiled data with conditions saved to: {output_path}")

        submit_button.on_click(on_submit_button_click)

        # Display all widgets
        display(widgets.VBox(list(identifier_widgets.values()) + [submit_button, output]))

    except Exception as e:
        print(f"Error: {e}")

# Example usage from the previous cell
output_filename = "combined_results.csv"
output_path = os.path.join(output_folder, output_filename)

# Call the function with the saved combined CSV file
map_conditions_and_save(output_path)


In [ ]:
# @title Plot individual data for each identifier

def plot_individual_data(output_folder, x_tick_interval=200):
    """
    Plot individual data for each identifier based on user-selected conditions and metrics.

    Parameters:
    - output_folder: Path where the concatenated CSV file is located and where plots will be saved.
    - x_tick_interval: Interval for x-axis labels.
    """
    # Locate the concatenated CSV file
    csv_files = glob.glob(os.path.join(output_folder, "**", "combined_results.csv"), recursive=True)

    if not csv_files:
        print(f"No concatenated CSV file found in the output folder: {output_folder}")
        return

    concatenated_csv = csv_files[0]
    print(f"Using concatenated file: {concatenated_csv}")

    # Read the concatenated CSV file
    data = pd.read_csv(concatenated_csv)

    # Extract unique conditions and identifiers
    #data["Identifier"] = data["CSV File Name"].str.extract(r'(^[A-Za-z0-9]+)_')
    # Extract identifiers like A4 or A4_1W from the file name
    data["Identifier"] = data["CSV File Name"].str.extract(r'^([A-Za-z0-9]+(?:_[0-9]+[A-Za-z])?)')
    unique_conditions = data.groupby("Identifier")["Condition"].first().reset_index()

    # Possible metrics to check in the CSV columns
    possible_metrics = {
        "Avg Velocity (µm/min)": "Avg Velocity",
        "Avg Direction (degrees)": "Avg Direction",
        "Max Flow Magnitude (µm/min)": "Max Flow Magnitude",
        "Divergence (1/min)": "Divergence"
    }

    # Filter metrics based on existing columns in the CSV
    available_metrics = {col: name for col, name in possible_metrics.items() if col in data.columns}

    # Create widgets for the available metrics
    metrics_checkboxes = {
        name: widgets.Checkbox(value=False, description=name, indent=False)
        for _, name in available_metrics.items()
    }

    # Ensure unique_conditions is sorted naturally by "Identifier"
    if "Identifier" in unique_conditions.columns:
        # Apply natural sorting to the DataFrame
        unique_conditions = unique_conditions.iloc[
            natsorted(unique_conditions.index, key=lambda i: unique_conditions.loc[i, "Identifier"])
        ]
    else:
        print("The column 'Identifier' does not exist in unique_conditions.")

    # Widgets for user input
    checkboxes = {}
    for _, row in unique_conditions.iterrows():  # Iterating through naturally sorted DataFrame
        identifier = row["Identifier"]
        condition = row["Condition"]
        checkboxes[identifier] = widgets.Checkbox(
            value=False,
            description=f"{identifier} - {condition}",
            indent=False
        )

    smoothing_window_widget = widgets.IntText(
        value=3,
        description="Smoothing Window:",
        style={'description_width': 'initial'}
    )

    graph_name_widget = widgets.Text(
        value="",
        description="Graph Name:",
        style={'description_width': 'initial'},
        placeholder="Enter custom name for the graph"
    )

    submit_button = widgets.Button(description="Plot Individual Data")
    output = widgets.Output()

    def plot_selected_individual(b):
        with output:
            clear_output()
            selected_identifiers = [key for key, checkbox in checkboxes.items() if checkbox.value]
            selected_metrics = [metric for metric, checkbox in metrics_checkboxes.items() if checkbox.value]
            smoothing_window = smoothing_window_widget.value
            custom_name = graph_name_widget.value.strip()

            if not selected_identifiers:
                print("No conditions selected for plotting.")
                return

            if not selected_metrics:
                print("No metrics selected for plotting.")
                return

            if not custom_name:
                print("Please enter a custom name for the graph.")
                return

            # Ensure the output folder exists
            os.makedirs(output_folder, exist_ok=True)

            for metric in selected_metrics:
                sanitized_metric = re.sub(r'[\^\w\s-]', '', metric.replace(" ", "_").lower())
                plot_filename_png = f"{metric.replace(' ', '_')}_{custom_name}.png"
                plot_filename_pdf = f"{metric.replace(' ', '_')}_{custom_name}.pdf"

                plt.figure(figsize=(12, 6))
                for identifier in selected_identifiers:
                    group_data = data[data["Identifier"] == identifier]

                    if group_data.empty:
                        print(f"No data found for identifier: {identifier}")
                        continue

                    time_labels = group_data["End Time Point (min)"]
                    metric_column = [col for col, name in available_metrics.items() if name == metric][0]
                    metric_values = group_data[metric_column]

                    smoothed_values = uniform_filter1d(metric_values, size=smoothing_window)

                    # Add condition to the label
                    condition = unique_conditions[unique_conditions["Identifier"] == identifier]["Condition"].values[0]
                    label = f"{identifier} - {condition}"

                    plt.plot(
                        time_labels,
                        smoothed_values,
                        marker='o',
                        label=label,
                        alpha=0.7
                    )

                # Dynamically include units in the y-label
                y_label = metric_column

                # Combine metric with custom name for the title
                plt.title(f"{metric} - {custom_name}", fontsize=16)
                plt.xlabel("Time (min)", fontsize=14)
                plt.ylabel(y_label, fontsize=14)
                plt.legend(loc="upper right", fontsize=12)
                plt.tight_layout()

                plot_path_png = os.path.join(output_folder, plot_filename_png)
                plot_path_pdf = os.path.join(output_folder, plot_filename_pdf)

                plt.savefig(plot_path_png, bbox_inches='tight')
                plt.savefig(plot_path_pdf, bbox_inches='tight')
                plt.show()
                print(f"Individual plot saved for {metric} - {', '.join(selected_identifiers)}: {plot_path_png} and {plot_path_pdf}")

    submit_button.on_click(plot_selected_individual)

    display(widgets.VBox(
        list(checkboxes.values()) +
        [widgets.Label("Select Metrics to Plot:")] +
        list(metrics_checkboxes.values()) +
        [smoothing_window_widget, graph_name_widget, submit_button, output]
    ))

plot_individual_data(output_folder=output_folder, x_tick_interval=200)

In [ ]:
# @title Plot averaged data by condition with error bars

def plot_averaged_data(output_folder, x_tick_interval=200):
    """
    Plot averaged data by condition with error bars based on user-selected metrics.

    Parameters:
    - output_folder: Path where the concatenated CSV file is located and where plots will be saved.
    - x_tick_interval: Interval for x-axis labels.
    """
    # Locate the concatenated CSV file
    csv_files = glob.glob(os.path.join(output_folder, "**", "combined_results.csv"), recursive=True)

    if not csv_files:
        print(f"No concatenated CSV file found in the output folder: {output_folder}")
        return

    concatenated_csv = csv_files[0]
    print(f"Using concatenated file: {concatenated_csv}")

    # Read the concatenated CSV file
    data = pd.read_csv(concatenated_csv)

    # Extract unique conditions and identifiers

    #data["Identifier"] = data["CSV File Name"].str.extract(r'(^[A-Za-z0-9]+)_')
    data["Identifier"] = data["CSV File Name"].str.extract(r'^([A-Za-z0-9]+(?:_[0-9]+[A-Za-z])?)')
    unique_conditions = data["Condition"].drop_duplicates()

    # Apply natural sorting to unique_conditions
    unique_conditions = natsorted(unique_conditions)

    # Possible metrics to check in the CSV columns
    possible_metrics = {
        "Avg Velocity (µm/min)": "Avg Velocity",
        "Avg Direction (degrees)": "Avg Direction",
        "Max Flow Magnitude (µm/min)": "Max Flow Magnitude",
        "Divergence (1/min)": "Divergence"
    }

    # Filter metrics based on existing columns in the CSV
    available_metrics = {col: name for col, name in possible_metrics.items() if col in data.columns}

    # Create widgets for the available metrics
    metrics_checkboxes = {
        name: widgets.Checkbox(value=False, description=name, indent=False)
        for _, name in available_metrics.items()
    }

    # Widgets for user input
    checkboxes = {}
    for condition in unique_conditions:
        checkboxes[condition] = widgets.Checkbox(
            value=False,
            description=condition,
            indent=False
        )

    smoothing_window_widget = widgets.IntText(
        value=3,
        description="Smoothing Window:",
        style={'description_width': 'initial'}
    )

    graph_name_widget = widgets.Text(
        value="",
        description="Graph Name:",
        style={'description_width': 'initial'},
        placeholder="Enter custom name for the graph"
    )

    submit_button = widgets.Button(description="Plot Averaged Data")
    output = widgets.Output()

    def plot_selected_averaged(b):
        with output:
            clear_output()
            selected_conditions = [key for key, checkbox in checkboxes.items() if checkbox.value]
            selected_metrics = [metric for metric, checkbox in metrics_checkboxes.items() if checkbox.value]
            smoothing_window = smoothing_window_widget.value
            custom_name = graph_name_widget.value.strip()

            if not selected_conditions:
                print("No conditions selected for plotting.")
                return

            if not selected_metrics:
                print("No metrics selected for plotting.")
                return

            if not custom_name:
                print("Please enter a custom name for the graph.")
                return

            # Ensure the output folder exists
            os.makedirs(output_folder, exist_ok=True)

            for metric in selected_metrics:
                sanitized_metric = re.sub(r'[\^\w\s-]', '', metric.replace(" ", "_").lower())
                plot_filename_png = f"{metric.replace(' ', '_')}_{custom_name}.png"
                plot_filename_pdf = f"{metric.replace(' ', '_')}_{custom_name}.pdf"

                plt.figure(figsize=(12, 6))
                grouped_data = data[data["Condition"].isin(selected_conditions)].groupby("Condition")

                for condition, group in grouped_data:
                    if group.empty:
                        print(f"No data found for condition: {condition}")
                        continue

                    # Preprocess: Convert the metric column to numeric
                    metric_column = [col for col, name in available_metrics.items() if name == metric][0]
                    group[metric_column] = pd.to_numeric(group[metric_column], errors='coerce')

                    # Drop rows with NaN values in the metric column
                    group = group.dropna(subset=[metric_column])

                    # Calculate mean and standard deviation for each time point
                    time_points = group["End Time Point (min)"].unique()
                    mean_values = group.groupby("End Time Point (min)")[metric_column].mean()
                    std_values = group.groupby("End Time Point (min)")[metric_column].std()

                    # Apply smoothing
                    smoothed_mean = uniform_filter1d(mean_values, size=smoothing_window)
                    smoothed_std = uniform_filter1d(std_values, size=smoothing_window)

                    # Plot data with error bars
                    plt.errorbar(
                        time_points,
                        smoothed_mean,
                        yerr=smoothed_std,
                        marker='o',
                        label=f"{condition}",
                        alpha=0.7
                    )

                # Dynamically include units in the y-label
                y_label = metric_column

                plt.title(f"{metric} - {custom_name}", fontsize=16)
                plt.xlabel("Time (min)", fontsize=14)
                plt.ylabel(y_label, fontsize=14)
                plt.legend(loc="upper right", fontsize=12)
                plt.tight_layout()

                plot_path_png = os.path.join(output_folder, plot_filename_png)
                plot_path_pdf = os.path.join(output_folder, plot_filename_pdf)

                plt.savefig(plot_path_png, bbox_inches='tight')
                plt.savefig(plot_path_pdf, bbox_inches='tight')
                plt.show()
                print(f"Averaged plot saved for {metric} - {', '.join(selected_conditions)}: {plot_path_png} and {plot_path_pdf}")

    submit_button.on_click(plot_selected_averaged)

    display(widgets.VBox(
        list(checkboxes.values()) +
        [widgets.Label("Select Metrics to Plot:")] +
        list(metrics_checkboxes.values()) +
        [smoothing_window_widget, graph_name_widget, submit_button, output]
    ))

plot_averaged_data(output_folder=output_folder, x_tick_interval=200)
